# Assignment
You were asked to analyse the last month of bike-sharing data in NYC (data source: https://citibikenyc.com/system-data ), and your manager thinks we could add insurance coverage for rides that last more than 30 minutes.

1. How many trips would be covered?

2. If your manager thinks we could charge 0.2 USD for each ride that takes longer than 30 minutes, how much revenue could we expect?

3. Your manager wants to understand the travel distance in distance buckets (0-1,2-4,4-9,10+). Please make a diagram.
 
Please present your analysis in a Jupyter notebook.

In [1]:
from ingest import download_zip_file, extract_zip_archive

In [2]:
file_name = "202402-citibike-tripdata.csv"
download_base_url = "https://s3.amazonaws.com/tripdata"

zip_archive_path = download_zip_file(file_name, download_base_url)

In [3]:
extract_zip_archive(zip_archive_path)